# pVCF to PLINK 2.0

> This notebook shows how to interact with genomic data in bed/bim/bam format using PLINK 2.0. We will learn how to convert between PLINK 1.x and PLINK 2.x file formats, merge variants from different chromosomes into a single file and filter them based on variant completeness and minor allelic frequencies (MAF). Please note the extended runtime of this notebook and that no subsequent analyses are contingent on its outputted files.

- runtime: 4hrs
- recommended instance: mem1_ssd1_v2_x16
- estimated cost: <£1.50

This notebook depends on:
* **PLINK install**


## List the exome sequences data directories in your project

Please note, that depending on your project's MTA the list of files might differ.

In [1]:
ls /mnt/project/Bulk/'Exome sequences'/

'Exome OQFE CRAM files'
'Exome OQFE variant call files (VCFs)'
'Population level exome OQFE variants, BGEN format - final release'
'Population level exome OQFE variants, BGEN format - interim 450k release'
'Population level exome OQFE variants, PLINK format - final release'
'Population level exome OQFE variants, PLINK format - interim 450k release'
'Population level exome OQFE variants, pVCF format - final release'
'Population level exome OQFE variants, pVCF format - interim 450k release'


## List the population variant files in PLINK 1.x (bed/bim/fam) format

In [1]:
ls -lah /mnt/project/Bulk/'Exome sequences'/'Population level exome OQFE variants, pVCF format - final release'/*c1_b1_*gz

-r--r--r-- 1 root root 26G Oct 14 14:12 '/mnt/project/Bulk/Exome sequences/Population level exome OQFE variants, pVCF format - interim 450k release/ukb23148_c1_b1_v1.vcf.gz'


### Install and test the PLINK2 binary
#### We recommend installing plink using the links available here:
https://www.cog-genomics.org/plink/2.0/

#### You can download the binary (AVX2 Intel; for example, using `wget <URL>`), before unzipping (`unzip <zip file>`) then making it exectutable (`chmod a+x <name>`)

#### if preferred, Plink is also available in the following locations:
https://anaconda.org/bioconda/plink2; https://github.com/chrchang/plink-ng

#### Once installed, continue with the below code chunks.


In [1]:
#test plink works
./plink2 --version

PLINK v2.00a6LM AVX2 Intel (3 Oct 2023)


### Next install and test BCFTOOLS
#### Following instructions here: http://samtools.github.io/bcftools/howtos/install.html, enter the following code (NB a large amount of text output will follow):

In [ ]:
git clone --recurse-submodules https://github.com/samtools/htslib.git
git clone https://github.com/samtools/bcftools.git
cd bcftools
autoheader && autoconf && ./configure --disable-libgsl --enable-perl-filters
make
cd ..
export BCFTOOLS_PLUGINS=$(pwd)/bcftools/plugins
p=$(pwd)/bcftools
PATH=$PATH:$p #set the path to the utility
bcftools --version

## Get reference genome

In [ ]:
wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/GRCh38_reference_genome/GRCh38_full_analysis_set_plus_decoy_hla.fa
wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/GRCh38_reference_genome/GRCh38_full_analysis_set_plus_decoy_hla.fa.fai

In [ ]:
#upload reference genome
dx mkdir ref_gen
dx upload GRCh38* --path ref_gen/

In [2]:
REF=`ls *fa`
echo $REF

GRCh38_full_analysis_set_plus_decoy_hla.fa


## Find pVCF path(s)

In [13]:
dx find data --brief --name ukb23157_c1_b1_v1.vcf.gz | xargs dx download

[===========================================================>] Completed 28,201,069,247 of 28,201,069,247 bytes (100%) /opt/notebooks/ukb23157_c1_b1_v1.vcf.gzz


In [6]:
VCF=`ls *vcf.gz`
echo $VCF

ukb23157_c1_b1_v1.vcf.gz


## Run bcftools normalization
This procedure left-aligns and normalizes indels, checks if REF alleles match the reference and split multiallelic sites into multiple rows. More info here: https://samtools.github.io/bcftools/bcftools.html#norm

In [ ]:
time bcftools norm -f $REF -m -any -Oz -o ${VCF%.*.*}.norm.vcf.gz $VCF #takes three hours

In [16]:
VCF=`ls *norm.vcf.gz`
echo $VCF

ukb23157_c1_b1_v1.norm.vcf.gz


## Make a Plink bed file

In [ ]:
./plink2 \
    --vcf $VCF \
    --vcf-idspace-to _ \
    --double-id \
    --allow-extra-chr 0 \
    --make-bed \
    --vcf-half-call m \
    --out "${VCF/.vcf.gz/""}"

## Convert the pVCF to PLINK 2.x formated dataset (pgen/pvar/psam)
PLINK 2.x formated files are faster to work with and have significntly smaller size than PLINK 1.x formated files.
However, PLINK 1.x is more popular format with wider support.

In [ ]:
time ./plink2 \
  --no-pheno \
  --vcf "$VCF" \
  --vcf-half-call 'haploid' \
  --make-pgen \
  --out "${VCF/.vcf.gz/""}"

## Convert to BED/BIM/FAM (PLINK 1.x format)

`--max-alleles` - excludes variants with more than the indicated value. When a variant has exactly one ALT allele and it's a missing-code, these filters treat it as having only one allele.
> see here: https://groups.google.com/g/plink2-users/c/rxMlVLIX-JA?pli=1 and https://github.com/meyer-lab-cshl/plinkQC/issues/10

In [ ]:
./plink2 \
  --no-pheno \
  --vcf "$VCF" \
  --vcf-half-call 'haploid' \
  --max-alleles 2 \
  --make-bed \
  --out test_vcf_bed

## Validate the output files

In [ ]:
./plink2 \
  --pfile "${VCF/.vcf.gz/""}" \
  --validate

In [ ]:
./plink2 \
  --bfile test_vcf_bed \
  --validate

In [ ]:
#upload file in case required
system("dx upload ukb23157_c1_b1_v1.norm.vcf.gz --path bed_maf/")